In [1]:
import math
import glob
import numpy as np
import pandas as pd
import scipy as sp

from scipy import stats
from scipy.fftpack import fft
from scipy.signal import medfilt
from scipy.fftpack import fft 
from scipy.fftpack import fftfreq 
from scipy.fftpack import ifft 
from numpy.fft import *
from scipy import fftpack


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split,SubsetRandomSampler, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold



from statistics import mean, stdev
import random
import csv
import time
import matplotlib.pyplot as plt
import itertools
import seaborn as sn
plt.style.use('ggplot')
sn.set_style("whitegrid")
%matplotlib inline

In [2]:
SAMPLING_FREQ = 20
WINDOW_SIZE = 2
OVERLAP = 20
NYQ = SAMPLING_FREQ / float(2) # Nyquist frequency 
CUTOFF = 0.3 
MAXFREQ = 10 
OUT_FEATURES = 3
DANCE_MOVES = ["jamesbond", "dab", "mermaid"]
TRAIN_SUBJECTS = ['chekjun','haritha', 'matthew' ,'nishanth', 'priyan']
SENSOR_COLS = ["acc_X", "acc_Y", "acc_Z", "gyro_X", "gyro_Y", "gyro_Z", "yaw", "pitch", "roll"]
DANCE_TO_NUM_MAP = {'dab': 0, 'jamesbond': 1, 'mermaid': 2}
NUM_TO_DANCE_MAP = {0: 'dab', 1: 'jamesbond', 2: 'mermaid'}
TRAIN_FILEPATH = "./capstone_data/train/*.csv"
TEST_FILEPATH = "./capstone_data/test/*.csv"

In [3]:
def load_data_paths(location):
    """
    Gets file path to each csv data file 
    Input: filepath to csv files 
    Returns: array of filepath to each csv which contains sensor data for each trial by a subject for a dance move 
    """
    data_paths = []
    for name in glob.glob(location):
        data_paths.append(name)
    return data_paths

In [4]:
def filter_signal(signal):
    """
    Applies 3rd order median filter for each signal i.e. Each axial column in dataset
    Input: numpy array 1D (one column)
    Returns: 3rd order median-filtered signal - numpy array 1D
    """
    array = np.array(signal)   
    med_filtered = medfilt(array, kernel_size=3) 
    return  med_filtered  

In [5]:
def mag_3_signals(x,y,z): 
    """
    Finding Euclidian magnitude of 3-axial signal 
    Inputs: x, y , z columns (numpy arrays)
    Returns: Euclidian magnitude of each row 
    """
    return [math.sqrt((x[i]**2+y[i]**2+z[i]**2)) for i in range(len(x))]

In [6]:
def t_domain_feature_per_signal(t_signal,cutoff,maxfreq,sampling_freq):
    """
    For each time-domain signal, i.e. accx,y,z and gyrox,y,z, get their respective time-domain components 
    Inputs: t_signal i.e. 1D numpy array (time domain signal), cutoff_freq, maxfreq, sampling_freq
    Returns: (total_component, t_DC_component , t_body_component, t_noise)
    """
    t_signal = np.array(t_signal)
    t_signal_length = len(t_signal) 
#     print("number of sample points in t_signal", t_signal_length)
    
    # 1D numpy array containing complex values
    f_signal = fft(t_signal) 
    
    # generate frequencies associated to f_signal complex values
    # frequency values between [-10hz:+10hz]
    freqs = np.array(sp.fftpack.fftfreq(t_signal_length, d = 1/float(sampling_freq))) 
    
    f_DC_signal = [] # DC_component in freq domain
    f_body_signal = [] # body component in freq domain 
    f_noise_signal = [] # noise in freq domain
    
    # iterate over all available frequencies
    for i, freq in enumerate(freqs):
          
        # selecting the f_signal value associated to freq
        value = f_signal[i]
        
        # Selecting DC_component values 
        if abs(freq) > cutoff:
            f_DC_signal.append(float(0))                                       
        else: 
            f_DC_signal.append(value) 
    
        # Selecting noise component values 
        if (abs(freq) <= maxfreq):
            f_noise_signal.append(float(0))  
        else:
            f_noise_signal.append(value) 

        # Selecting body_component values 
        if (abs(freq) <= cutoff or abs(freq) > maxfreq):
            f_body_signal.append(float(0))
        else:
            f_body_signal.append(value) 
    
   
    t_DC_component = ifft(np.array(f_DC_signal)).real
    t_body_component = ifft(np.array(f_body_signal)).real
    t_noise = ifft(np.array(f_noise_signal)).real
    
    # extracting the total component(filtered from noise)
    total_component = t_signal - t_noise  
                                     
    return (total_component,t_DC_component,t_body_component,t_noise)

In [7]:
def time_domain_feature_gen(df, cutoff, maxfreq, sampling_freq):
    """
    For each trial of a dance move by a subject, generate a df containing time domain features and update the time_sig_dic
    Input : df i.e. df containing 1 min readings of 6 axial data of each trial of a dance move 
    Input : cutoff
    Input : maxfreq
    Input : name_for_df, i.e. key to represent the df for which time domain features were generated 
    Input : sampling_freq
    Returns : dframe with 9 axial values generated from raw data 
    """
    time_sig = {}
    
    # iterate through all six axial signals 
    for column in df.columns:
        t_signal = np.array(df[column])
        medfiltered_sig = filter_signal(t_signal)
        
        if 'acc' in column: 
            _,grav_acc,body_acc,_ = t_domain_feature_per_signal(medfiltered_sig,cutoff,maxfreq,sampling_freq) 
            time_sig['t_body_'+ column] = body_acc
            time_sig['t_grav_'+ column] = grav_acc 
            
        elif 'gyro' in column: 
            _,_,body_gyro,_ = t_domain_feature_per_signal(medfiltered_sig,cutoff,maxfreq, sampling_freq)
            time_sig['t_body_gyro_'+ column[-1]] = body_gyro
    
    
    # all 9 axial signals generated above are reordered to facilitate find magnitude
    new_columns_ordered = ['t_body_acc_X','t_body_acc_Y','t_body_acc_Z',
                          't_grav_acc_X','t_grav_acc_Y','t_grav_acc_Z',
                          't_body_gyro_X','t_body_gyro_Y','t_body_gyro_Z']
    
    
    ordered_time_sig_df = pd.DataFrame()
    for col in new_columns_ordered: 
        ordered_time_sig_df[col] = time_sig[col] 
    
    # Calculating magnitude by iterating over each 3-axial signal
    for i in range(0,9,3): 
        mag_col_name = new_columns_ordered[i][:-1]+'mag'
        x_col = np.array(ordered_time_sig_df[new_columns_ordered[i]])   # copy X_component
        y_col = np.array(ordered_time_sig_df[new_columns_ordered[i+1]]) # copy Y_component
        z_col = np.array(ordered_time_sig_df[new_columns_ordered[i+2]]) # copy Z_component
        
        mag_signal = mag_3_signals(x_col,y_col,z_col) # calculate magnitude of each signal[X,Y,Z]
        ordered_time_sig_df[mag_col_name] = mag_signal 
    
    return ordered_time_sig_df 

In [8]:
def generate_final_dataset(raw_dic, cutoff, maxfreq, sampling_freq, mapping_dic):
    """
    Get final processed data for segmentation 
    Input: raw_dic i.e. raw_test or raw_train dict which contains each {subjectName}_{dance}_{trialNum}'s sensor data
    Inputs: cutoff, maxfreq, sampling_freq
    Input : mapping_dic i.e. to map dancemove to target as an integer, DANCE to NUM
    Returns: final_dic with the keys as {subjectName}_{dance}_{trialNum} with processed data that has 16 cols 
    """
    final_dic = {}
    for key in raw_dic.keys():
        df = time_domain_feature_gen(raw_dic[key].drop(["subject", "trialNum", "dance"], axis = 1), cutoff, maxfreq, sampling_freq)
        sub = np.unique(raw_dic[key]["subject"])
        trial = np.unique(raw_dic[key]["trialNum"])
        dancemove = np.unique(raw_dic[key]["dance"])
        df["subject"] = raw_dic[key]["subject"]
        df["trialNum"] = raw_dic[key]["trialNum"]
        df["dance"] = raw_dic[key]["dance"]
        df["target"] = df["dance"].map(mapping_dic)
        final_dic[f"{sub}_{dancemove}_{trial}"] = df
        
    return final_dic

In [9]:
def gen_mapping(danceArray):
    """
    Get two dicts. One with dance mapped to number and the other with number mapped to dance 
    Input: Unique dance moves in 1d array
    Returns: ({dance: num}, {num: dance})
    """
    map_dance_to_num = {}
    map_num_to_dance = {}
    for i, move in enumerate(danceArray): 
        map_num_to_dance[i] = move
        map_dance_to_num[move] = i
    return (map_dance_to_num, map_num_to_dance)

In [10]:
def getInputVector(reshapedSegments, samplingFreq, window, numOfAxis):
    input_features = samplingFreq * window * numOfAxis
    inputVector = reshapedSegments.reshape(reshapedSegments.shape[0], input_features)
    return inputVector.astype("float32")

In [11]:
def segmentation(df,samplingFreq,window,overlap,encodedTargetColumn, axisCount):
    """
    Inputs: df, samplingFreq, window,overlap, encodedTargetColumn
    Note : window = the time interval of one window in seconds,
    overlap = num of steps to take from one segment/window to the next i.e. if 50% => 50 steps 
    """
#     window size = nrows = Sampling freq(Hz) * window(secs)
#     if overlap = nrows, then there is no overlap bewteen segments,
#     if accx,y,z and gyrox,y,z => 6 as numOfAxis. 
    numOfAxis = axisCount
    segments = []
    labels = []
    nrows = samplingFreq * window
    for i in range(0, len(df) - nrows, overlap): 
        bax = df["t_body_acc_X"].values[i:i+nrows]
        bay = df["t_body_acc_Y"].values[i:i+nrows]
        baz = df["t_body_acc_Z"].values[i:i+nrows]
        gx = df["t_grav_acc_X"].values[i:i+nrows]
        gy = df["t_grav_acc_Y"].values[i:i+nrows]
        gz = df["t_grav_acc_Z"].values[i:i+nrows]
        bgx = df["t_body_gyro_X"].values[i:i+nrows]
        bgy = df["t_body_gyro_Y"].values[i:i+nrows]
        bgz = df["t_body_gyro_Z"].values[i:i+nrows]
        bam = df["t_body_acc_mag"].values[i:i+nrows]
        gam = df["t_grav_acc_mag"].values[i:i+nrows]
        bgm = df["t_body_gyro_mag"].values[i:i+nrows]
        # retrieve the most used label in this segment 
        label = stats.mode(df[encodedTargetColumn][i:i+nrows])[0][0]
        # each segment appended represents a window's values for each of 
        # the axial values
        segments.append([bax,bay,baz,gx,gy,gz,bgx,bgy,bgz,bam,gam,bgm])
        labels.append(label)
    
    reshaped_segments = np.asarray(segments,dtype =np.float32).reshape(-1,nrows,numOfAxis)
    labels = np.asarray(labels)
    
    # reshaped_segments will be x and labels will be y 
    return reshaped_segments, labels

In [12]:
def lookUp(dframe,sub,trialNum, dance):
    """
    Lookup a particular subject's values based on trialNum and dance in the raw dataframe 
    Inputs: dataframe, str(subject), str(trialNumber) and str(danceMove)
    Returns: the dataframe under consideration.
    """
    df_considered = dframe[(dframe["subject"] == sub) & (dframe["trialNum"] == trialNum) & (dframe["dance"] == dance)]
    return df_considered


In [13]:
def normaliseData(dframe, columnNames):
    """
    Normalize features for training data set (values between 0 and 1).Columns rounded to 4dp after normalisation
    Inputs: dframe and the list of columns to be normalised
    No return value
    """
    pd.options.mode.chained_assignment = None  # default='warn'
    for col in columnNames:
        dframe[col] = dframe[col] / dframe[col].max()
        dframe[col] = dframe[col].round(4)

In [14]:
def gen_rawData(given_filepaths):
    """
    Generate training and test dataframes from raw sensor data 
    Input: given_filepaths: filepaths 1D array
    Returns: dictionary of raw dfs, with key being {subjectName}_{dance}_{trialNum}
    """
    frames = {}
    # each filepath corresponds to a diff file 
    # each file has 1200 values, each subject does a dance move for 3 times 
    # hence 3600 values per subject for a dance move 
    # thus for n dance moves, each subject has 3600 * n values 
    # with k subjects, the dataset will have k * 3600 * n values 
    for filepath in given_filepaths:
        _, s, subjectName, ext = filepath.split("_")
        _, _, dance = s.split("/")
        trialNum, _ = ext.split(".")
        raw_df = pd.read_csv(filepath, names=SENSOR_COLS, index_col=None)
        raw_df.dropna(inplace= True)
        raw_df.drop(["yaw","pitch","roll"], axis=1, inplace=True)
        raw_df.reset_index(drop=True,inplace=True)
        for col in raw_df.columns:
            raw_df[col] = raw_df[col].div(100).round(6)
        normaliseData(raw_df, raw_df.columns)
        raw_df["subject"] = subjectName
        raw_df["trialNum"] = int(trialNum)
        raw_df["dance"] = dance
#         print(raw_df.shape)
#         print(raw_df.head(3))
        frames[f"{subjectName}_{dance}_{trialNum}"] = raw_df
    return frames 

In [15]:
raw_test_dic = gen_rawData(load_data_paths(TEST_FILEPATH))
raw_train_dic = gen_rawData(load_data_paths(TRAIN_FILEPATH))

training_dic = generate_final_dataset(raw_train_dic,CUTOFF,MAXFREQ, SAMPLING_FREQ, DANCE_TO_NUM_MAP)
testing_dic = generate_final_dataset(raw_test_dic,CUTOFF,MAXFREQ, SAMPLING_FREQ, DANCE_TO_NUM_MAP)
test_df = pd.concat(testing_dic.values(), axis = 0, ignore_index=True)
train_df = pd.concat(training_dic.values(), axis = 0, ignore_index=True)
data_train, lbl_train = segmentation(train_df, SAMPLING_FREQ, WINDOW_SIZE, OVERLAP, "target", 12 )
data_test, lbl_test  = segmentation(test_df,SAMPLING_FREQ, WINDOW_SIZE, OVERLAP, "target", 12)
# raw_train = pd.concat(raw_train_dic.values(), axis = 0,  ignore_index=True)
# raw_train["target"] = raw_train["dance"].map(DANCE_TO_NUM_MAP)
# raw_test["target"] = raw_test["dance"].map(DANCE_TO_NUM_MAP)
# lookUp(raw_test, "sean", 3, "jamesbond")
# dance_to_num , num_to_dance = gen_mapping(np.unique(raw_train["dance"]))

In [16]:
training_X = getInputVector(data_train, SAMPLING_FREQ, WINDOW_SIZE, 12)
training_X.shape

(2698, 480)

In [17]:
print(lbl_train.shape)
len(training_X)

(2698,)


2698

In [18]:
testing_X = getInputVector(data_test, SAMPLING_FREQ, WINDOW_SIZE, 12)
testing_X.shape

(538, 480)

In [19]:
lbl_test.shape

(538,)

In [20]:
class Model(nn.Module):
    def __init__(self, in_features, h1, h2, out_features=OUT_FEATURES):
        super().__init__()
        self.fc1 = nn.Linear(in_features,h1)    # input layer
        self.fc2 = nn.Linear(h1, h2)            # hidden layer
        self.out = nn.Linear(h2, out_features)  # output layer
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x
    

# Instantiate the Model class using parameter defaults:
torch.manual_seed(32)
hidden_layer_1_nodes = 80
hidden_layer_2_nodes = 40
mlp = Model(in_features=training_X.shape[1], h1=hidden_layer_1_nodes, h2=hidden_layer_2_nodes)
X_train = torch.FloatTensor(training_X)
X_test = torch.FloatTensor(testing_X)
y_train = torch.LongTensor(lbl_train)
y_test = torch.LongTensor(lbl_test)
# trainloader = DataLoader(X_train, batch_size=60, shuffle=True)
# testloader = DataLoader(X_test, batch_size=60, shuffle=False)
# len(X_important_train[0])


k = 10       
skfcv = StratifiedKFold(n_splits=k, shuffle=True, random_state=1)
epochs = 100
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.01)
training_loss = {}
train_batch_size = 108
test_batch_size = 72
val_acc = []
# per fold 
for fold, (train_index, test_index) in enumerate(skfcv.split(X_train, y_train)):
    x_train_fold, x_test_fold = X_train[train_index], X_train[test_index]
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]
    train_combined = TensorDataset(x_train_fold, y_train_fold)
    test_combined = TensorDataset(x_test_fold, y_test_fold)
    trainloader = DataLoader(train_combined, batch_size=train_batch_size, shuffle=True)
    testloader = DataLoader(test_combined, batch_size=test_batch_size, shuffle=False)
    # per epoch
    for i in range(epochs): 
        i+=1
        # per batch 
        losses = [] 
        val_correct_preds = 0
        count = 0 
        for batch_idx, (data, target) in enumerate(trainloader):
            mlp.train()
             
            # training 
            y_pred = mlp.forward(data)
            loss = criterion(y_pred, target)
            losses.append(loss)
            
            # backtracking 
            optimizer.zero_grad()
            loss.backward()
            optimizer.step() 
        
        # validating 
        with torch.no_grad():
            mlp.eval()
            for val_batch_idx, (val_data, val_target) in enumerate(testloader):
                y_out = mlp.forward(val_data)
#                 print(y_out.shape[0])
                for row in range(y_out.shape[0]):
#                     print("predicted:" , y_out[row].argmax())
#                     print("actual: ", val_target[row])
                    if y_out[row].argmax() == val_target[row]:
                        val_correct_preds += 1
                    count += 1

            
    # per fold         
    with torch.no_grad():
        training_loss[fold] = np.array(losses).mean()
        print("-----------------------")
        print(f"fold: {fold} , training_loss: {training_loss[fold]}")
        print(f"fold: {fold}, {val_correct_preds} out of {count} = {100*val_correct_preds/count:.2f}% correct")
        print("-----------------------")
        val_acc.append(100*val_correct_preds/count)
        count = 0

print()
print("Done Training")
print("Max Validation Accuracy: ",  np.array(val_acc).max())

-----------------------
fold: 0 , training_loss: 3.7776148928969633e-06
fold: 0, 266 out of 270 = 98.52% correct
-----------------------
-----------------------
fold: 1 , training_loss: 7.673599611734971e-06
fold: 1, 267 out of 270 = 98.89% correct
-----------------------
-----------------------
fold: 2 , training_loss: 4.406187144923024e-06
fold: 2, 268 out of 270 = 99.26% correct
-----------------------
-----------------------
fold: 3 , training_loss: 2.9104517125233542e-06
fold: 3, 268 out of 270 = 99.26% correct
-----------------------
-----------------------
fold: 4 , training_loss: 2.229790652563679e-06
fold: 4, 270 out of 270 = 100.00% correct
-----------------------
-----------------------
fold: 5 , training_loss: 4.502440162923449e-07
fold: 5, 270 out of 270 = 100.00% correct
-----------------------
-----------------------
fold: 6 , training_loss: 1.1776293717957742e-07
fold: 6, 270 out of 270 = 100.00% correct
-----------------------
-----------------------
fold: 7 , training

In [21]:
# TO EVALUATE THE ENTIRE TEST SET
with torch.no_grad():
    start_time = time.time()
    y_val = mlp.forward(X_test)
    print("--- %s execution time in seconds ---" % (time.time() - start_time))
    loss = criterion(y_val, y_test)
    print(f'Loss with test set : {loss:.8f}')

--- 0.01548004150390625 execution time in seconds ---
Loss with test set : 1.58633745


In [22]:

with torch.no_grad():
    preds = []
    correct = 0
    for i,data in enumerate(X_test):
        y_val = mlp.forward(data)
#         print(f'{i+1:2}. {str(y_val):38}  {y_test[i]}')
#         print(y_val.argmax().item(),y_test[i], y_val.argmax() )
        preds.append(y_val.argmax().item())
        if y_val.argmax().item() == y_test[i]:
            correct += 1
    print(f'\n{correct} out of {len(y_test)} = {100*correct/len(y_test):.2f}% correct')
    y_preds = torch.tensor(preds, dtype = torch.int64)
    stacked = torch.stack((y_test,y_preds),dim=1)
#     print(stacked.shape)
#     print(stacked)
    cmt = torch.zeros(OUT_FEATURES,OUT_FEATURES, dtype=torch.int64)
    for p in stacked:
        tl, pl = p.tolist()
#         print(tl,pl)
        cmt[tl, pl] = cmt[tl, pl] + 1   
#     print(cmt)


469 out of 538 = 87.17% correct


In [23]:
print("Classification Report for MLP :")
print(classification_report(y_test, y_preds))

Classification Report for MLP :
              precision    recall  f1-score   support

           0       0.76      0.93      0.84       182
           1       0.92      0.78      0.85       180
           2       0.99      0.90      0.94       176

    accuracy                           0.87       538
   macro avg       0.89      0.87      0.87       538
weighted avg       0.89      0.87      0.87       538



In [24]:
# model weights 
with torch.no_grad():
    mlp_params = {}
    for name, param in mlp.named_parameters():
#         print(name,param)
        mlp_params[name] = param.numpy().copy().tolist()
print(mlp_params.keys())
for key in mlp_params.keys(): 
    print(f"{key} : {len(mlp_params[key])} neurons in {key} layer")
    print(f" number of connections  : {len(mlp_params[key])} * {np.asarray(mlp_params[key][len(mlp_params[key]) -1]).size} = {len(mlp_params[key]) * np.asarray(mlp_params[key][len(mlp_params[key]) -1]).size } ")
    
    
torch.save(mlp.state_dict(), 'MLPW9.pt')

loaded_model = Model(in_features=len(training_X[1]),h1=80, h2=40) 
loaded_model.load_state_dict(torch.load('MLPW9.pt'))
loaded_model.eval()


with torch.no_grad():
    y_val = loaded_model.forward(X_test)
    loss = criterion(y_val, y_test)
print(f'{loss:.8f}')

random.seed(99)
random_int = random.randint(0, len(testing_X))
random_input = torch.FloatTensor(testing_X[random_int])
print(random_input)

with torch.no_grad():
    print(loaded_model(random_input))
    print()
    print(loaded_model(random_input).max())
    print(loaded_model(random_input).argmax().item())
    print(f"Predicted Output: {NUM_TO_DANCE_MAP[loaded_model(random_input).argmax().item()]}")
    print(f"Actual Output: {NUM_TO_DANCE_MAP[y_test[random_int].item()]}")

dict_keys(['fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'out.weight', 'out.bias'])
fc1.weight : 80 neurons in fc1.weight layer
 number of connections  : 80 * 480 = 38400 
fc1.bias : 80 neurons in fc1.bias layer
 number of connections  : 80 * 1 = 80 
fc2.weight : 40 neurons in fc2.weight layer
 number of connections  : 40 * 80 = 3200 
fc2.bias : 40 neurons in fc2.bias layer
 number of connections  : 40 * 1 = 40 
out.weight : 3 neurons in out.weight layer
 number of connections  : 3 * 40 = 120 
out.bias : 3 neurons in out.bias layer
 number of connections  : 3 * 1 = 3 
1.58633745
tensor([-0.6312, -0.6898, -0.6930, -0.6803, -0.6045, -0.3790, -0.2639, -0.1567,
        -0.0339,  0.0099,  0.2816,  0.7656,  0.8555,  0.8661,  0.6636,  0.5470,
         0.4613,  0.3828,  0.0358, -0.3751, -0.3695, -0.5615, -0.3293, -0.3262,
        -0.3242, -0.2915, -0.6062, -0.3550, -0.3566,  0.0739,  0.5351,  0.9480,
         1.0689,  1.0629,  0.8435,  0.6235,  0.2452, -0.1810, -0.1904, -0.5704,
        

In [25]:
with open('mlp.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in mlp_params.items():
       writer.writerow([key, value])

In [26]:
with torch.no_grad(): 
    x = X_test.numpy().copy().tolist()
    np.savetxt("X_test.csv", x, delimiter=",")
    y = y_test.numpy().copy().tolist()
    np.savetxt("y_test.csv", y, delimiter=",")